# Get Daily Station Data from xmACIS
## Written By Jared Rennie (@jjrennie)

Taps into the ACIS API to get daily station data for its period of record.

- API: https://www.rcc-acis.org/docs_webservices.html
- Query Builder: https://builder.rcc-acis.org/

### What You Need

First off, the entire codebase works in Python 3. In addition to base Python, you will need the following packages installed: 
- requests (to access the api)
- pandas (to slice annd dice the data)
    
The "easiest" way is to install these is by installing <a href='https://www.anaconda.com' target="_blank">anaconda</a>, and then applying <a href='https://conda-forge.org/' target="_blank">conda-forge</a>. Afterward, then you can install the above packages. 

### Importing Packages
Assuming you did the above, it should (in theory) import everything no problem:

In [1]:
# Import Packages
import json,requests,sys
import pandas as pd

print("SUCCESS!")

SUCCESS!


If you made it this far, great!

### Insert Arguments
To access the data, we will be utilizing the <a href='https://www.rcc-acis.org/docs_webservices.html' target='blank'>ACIS API</a>, which is a quick and easy way to access our station data without having to download it yourself. Now we need to know what station to get data for. The ACIS API accepts all sorts of IDs, including:

- FAA (i.e. AVL)
- ghcn (i.e. USW00003812)
- ThreadEx (i.e. AVLthr)

If you're not sure, you can refer to the API documentation above.

 **Change the arguments below to your liking**

In [2]:
# Insert Arguments Here
stationID = 'DCA'

The rest of the code should work without making any changes to it, but if you're interested, keep on reading to see how the sausage is made.

This next block of code will attempt to access the data we want from the ACIS API. The API is publicly available, but sometimes there are hiccups when getting the data. We tried to account for this with a try/exept in this code block and it will let you know if it fails after 3 seconds. If this happens, wait a minute, then try again.

In [3]:
# DEFINE ACIS URL 
# API Info: https://www.rcc-acis.org/docs_webservices.html
# Query Builder: https://builder.rcc-acis.org/
acis_url = 'http://data.rcc-acis.org/StnData'

# Build JSON to access ACIS API
# Gets 5 Variables (MaxT, MinT, Precip, Snow, Snow Depth)
# To get more, use the API Info or Query Builder above.
payload = {
"output": "json",
"params": {"elems":[{"name":"maxt","interval":"dly","prec":1},
                    {"name":"mint","interval":"dly","prec":1},
                    {"name":"pcpn","interval":"dly","prec":2},
                    {"name":"snow","interval":"dly","prec":1},
                    {"name":"snwd","interval":"dly","prec":1}],
           "sid":stationID,
           "sdate":"por",
           "edate":"por"
          } 
}
# Make Request
try:
    r = requests.post(acis_url, json=payload,timeout=3)
    acisData = r.json()
    print("SUCCESS!")
except Exception as e:
	sys.exit('\nSomething Went Wrong With Accessing API after 3 seconds, Try Again')

SUCCESS!


If it says "SUCCESS!" then congrats you got the data!

### Let's check the data!
How does it look? Well the data comes back as a JSON, which can be a little confusing to look at, so let's extract the information we need, and reorganize it a bit.

First, the JSON has a 'meta' key and a 'data' key. The 'meta' key gets us info like station name, latitude, longitude, etc. And 'data' is the actual data we requested. So let's get some station info, and convert the data into a pandas dataframe, which makes it easier to see. 

In [4]:
# Get Station Info
stationName=acisData['meta']['name'].title()
stationState=acisData['meta']['state']

# Convert data into Pandas DataFrame
acisPandas = pd.DataFrame(acisData['data'], columns=['DATE','TMAX','TMIN','PRCP','SNOW','SNWD'])

print("\nSuccessfully Orgainzed Data for: ",stationName,',',stationState)
print(acisPandas)


Successfully Orgainzed Data for:  Washington Reagan National Airport , VA
             DATE  TMAX  TMIN  PRCP SNOW SNWD
0      1941-06-16  78.0  59.0  0.00  0.0  0.0
1      1941-06-17  77.0  57.0  0.16  0.0  0.0
2      1941-06-18  83.0  55.0  0.00  0.0  0.0
3      1941-06-19  89.0  65.0  0.00  0.0  0.0
4      1941-06-20  92.0  67.0  0.00  0.0  0.0
...           ...   ...   ...   ...  ...  ...
30546  2025-02-01  53.0  32.0     T  0.0  0.0
30547  2025-02-02  40.0  27.0  0.00  0.0  0.0
30548  2025-02-03  53.0  33.0  0.00  0.0  0.0
30549  2025-02-04  64.0  40.0  0.00  0.0  0.0
30550  2025-02-05  41.0  34.0  0.05  0.0  0.0

[30551 rows x 6 columns]


Sometimes people want to know what the station's period of record is, so let's get that info. 

In [5]:
stationStart=acisPandas.iloc[[0]]['DATE'].values[0][0:4]
stationEnd=acisPandas.iloc[[-1]]['DATE'].values[0][0:4]
print("Period of Record: ",stationStart,"-",stationEnd)

Period of Record:  1941 - 2025


### Output to CSV
Last step outputs the data to a CSV file so you can load it and play with it elsewhere. Or you can keep playing with it here. You do you!

In [6]:
# Send to CSV
outFile='./'+stationID+'_Daily_POR.csv'
acisPandas.to_csv(outFile,index=False)

# If you made it this far. Success!
print('SUCESSFULLY GOT DATA AND PUT IN:',outFile)

SUCESSFULLY GOT DATA AND PUT IN: ./DCA_Daily_POR.csv


That's it! Now you have data for 1 station in the database. What if you want to get multiple stations? It's doable!

**Congrats on completing this notebook! Now go forth and get your data!**